# Vítej!

Tento dokument obsahuje zdrojové kódy animací ke druhé sérii seriálu KSP. Před spouštěním opět nezapomeň Manim importovat spuštěním následujícího řádku!

In [ ]:
from manim import *

## Práce se skupinami objektů

In [ ]:
%%manim -v WARNING -qh VGroupExample


class VGroupExample(Scene):
    def construct(self):
        s1 = Square(color=RED)
        s2 = Square(color=GREEN)
        s3 = Square(color=BLUE)

        s1.next_to(s2, LEFT)
        s3.next_to(s2, RIGHT)

        self.play(Write(s1), Write(s2), Write(s3))

        group = VGroup(s1, s2, s3)

        # aplikace škálování na celou skupinu
        self.play(group.animate.scale(1.5).shift(UP))

        # na skupině můžeme indexovat
        self.play(group[1].animate.shift(DOWN * 2))

        # změna barvy se aplikuje na všechny objekty
        self.play(group.animate.set_color(WHITE))
        self.play(group.animate.set_fill(WHITE, 1))

In [ ]:
%%manim -v WARNING -qh ArrangeExample

from random import seed, uniform


class ArrangeExample(Scene):
    def construct(self):
        seed(0xDEADBEEF)

        # používáme *, protože VGroup bere samotné objekty (viz minulý příklad)
        circles = VGroup(
            *[
                Circle(radius=0.1)
                .scale(uniform(0.5, 4))
                .shift(UP * uniform(-3, 3) + RIGHT * uniform(-5, 5))
                for _ in range(12)
            ]
        )

        self.play(FadeIn(circles))

        # uspořádání vedle sebe
        self.play(circles.animate.arrange())

        # různé odsazení a směry
        self.play(circles.animate.arrange(direction=DOWN, buff=0.1))
        self.play(circles.animate.arrange(buff=0.4))

In [ ]:
%%manim -v WARNING -qh ArrangeInGridExample

from random import seed, uniform


class ArrangeInGridExample(Scene):
    def construct(self):
        seed(0xDEADBEEF)

        circles = VGroup(
            *[
                Circle(radius=0.1)
                .scale(uniform(0.5, 2))
                .shift(UP * uniform(-3, 3) + RIGHT * uniform(-5, 5))
                for _ in range(9 ** 2)
            ]
        )

        self.play(FadeIn(circles))

        # uspořádání do mřížky
        self.play(circles.animate.arrange_in_grid())

        # různé odsazení a počet řádků/sloupců
        self.play(circles.animate.arrange_in_grid(rows=5, buff=0))
        self.play(circles.animate.arrange_in_grid(cols=12, buff=0.3))

## Přidávání a odebírání objektů

In [ ]:
%%manim -v WARNING -qh AddRemoveExample


class AddRemoveExample(Scene):
    def construct(self):
        square = Square(fill_color=WHITE, fill_opacity=1)
        small_scale = 0.6

        triangle = Triangle(fill_opacity=1).scale(small_scale).move_to(square)

        self.play(Write(square))

        # přidání trojúhelníku pod čtverec
        self.bring_to_back(triangle)
        self.play(square.animate.shift(LEFT * 2))

        circle = Circle(fill_opacity=1).scale(small_scale).move_to(square)

        # přidání kruhu pod čtverec
        self.bring_to_back(circle)
        self.play(square.animate.shift(RIGHT * 2))

        square2 = (
            Square(stroke_color=GREEN, fill_color=GREEN, fill_opacity=1)
            .scale(small_scale)
            .move_to(square)
        )
        
        self.remove(triangle)
        
        # stejné jako self.add
        # přidání dopředu tu spíš nechceme, ale je dobré vidět co dělá
        self.bring_to_front(square2)

        self.play(square.animate.shift(RIGHT * 2))

## Překrývající-se animace

In [ ]:
%%manim -v WARNING -qh AnimationGroupExample


class AnimationGroupExample(Scene):
    def construct(self):
        c1 = Square(color=RED)
        c2 = Square(color=GREEN)
        c3 = Square(color=BLUE)

        VGroup(c1, c2, c3).arrange(buff=1)

        # každá další animace se spustí v polovině té předchozí (0.5)
        self.play(AnimationGroup(Write(c1), Write(c2), Write(c3), lag_ratio=0.5))

        # každá další animace se spustí v desetině té předchozí (0.1)
        self.play(AnimationGroup(FadeOut(c1), FadeOut(c2), FadeOut(c3), lag_ratio=0.1))

        # jedna z animací může být rovněž skupina, která může mít sama o sobě zpoždění
        self.play(
            AnimationGroup(
                AnimationGroup(Write(c1), Write(c2), lag_ratio=0.1),
                Write(c3),
                lag_ratio=0.5,
            )
        )

        # lag_ratio může být i záporné (animace se budou spouštět obráceně)
        self.play(AnimationGroup(FadeOut(c1), FadeOut(c2), FadeOut(c3), lag_ratio=-0.1))

In [ ]:
%%manim -v WARNING -qh VGroupLagRatioExample


class VGroupLagRatioExample(Scene):
    def construct(self):
        squares = VGroup(Square(), Square(), Square()).arrange(buff=0.5).scale(1.5)
        
        # postupné vykreslení čtverců
        self.play(Write(squares))

        # FadeOut lag_ratio má nulové, animace se vykonají najednou
        self.play(FadeOut(squares))

        squares.set_color(BLUE)
        
        # lag_ratio můžeme manuálně nastavit tak, aby se čtverce vykreslily najednou        
        self.play(Write(squares, lag_ratio=0))

        self.play(FadeOut(squares, lag_ratio=0.5))

## Práce s pozorností

In [ ]:
%%manim -v WARNING -qh AttentionExample


class AttentionExample(Scene):
    def construct(self):
        c1 = Square()

        labels = [
            Tex("Flash"),
            Tex("Indicate"),
            Tex("Wiggle"),
            Tex("FocusOn"),
            Tex("Circumscribe"),
        ]

        # labely posuneme dolů (pod čtverec)
        for label in labels:
            label.shift(DOWN * 1.5).scale(1.5)

        def switch_labels(i: int):
            """Animace přeměny jednoho labelu na druhého."""
            return AnimationGroup(
                FadeOut(labels[i], shift=UP * 0.7),
                FadeIn(labels[i + 1], shift=UP * 0.7),
            )

        self.play(Write(c1))

        self.play(FadeIn(labels[0], shift=UP * 0.5), c1.animate.shift(UP))

        # Flash
        self.play(Flash(c1, flash_radius=1.6, num_lines=20))

        # Indicate
        self.play(AnimationGroup(switch_labels(0), Indicate(c1), lag_ratio=0.7))

        # Wiggle
        self.play(AnimationGroup(switch_labels(1), Wiggle(c1), lag_ratio=0.7))

        # FocusOn
        self.play(AnimationGroup(switch_labels(2), FocusOn(c1), lag_ratio=0.7))

        # Circumscribe
        self.play(AnimationGroup(switch_labels(3), Circumscribe(c1), lag_ratio=0.7))

## Transformace

In [ ]:
%%manim -v WARNING -qh BasicTransformExample


class BasicTransformExample(Scene):
    def construct(self):
        c = Circle().scale(2)
        s = Square().scale(2)

        self.play(Write(c))

        self.play(Transform(c, s))

In [ ]:
%%manim -v WARNING -qh BadTransformExample


class BadTransformExample(Scene):
    def construct(self):
        good = [Circle(color=GREEN), Square(color=GREEN), Triangle(color=GREEN)]
        bad = [Circle(color=RED), Square(color=RED), Triangle(color=RED)]

        # uspořádání do mřížky - nahoře dobré, dole špatné
        VGroup(*(good + bad)).arrange_in_grid(rows=2, buff=1)

        self.play(Write(good[0]), Write(bad[0]))

        self.play(
            Transform(good[0], good[1]),
            Transform(bad[0], bad[1]),
        )

        self.play(
            Transform(good[0], good[2]),
            Transform(bad[1], bad[2]),
        )

In [ ]:
%%manim -v WARNING -qh TransformMatchingShapesExample


class TransformMatchingShapesExample(Scene):
    def construct(self):
        ksp_matching = Tex("KSP").scale(5)
        ksp_full_matching = Tex("Korespondenční Seminář z Programování")

        ksp_regular = ksp_matching.copy().set_color(BLUE)
        ksp_full_regular = ksp_full_matching.copy().set_color(BLUE)

        VGroup(ksp_matching, ksp_regular).arrange(direction=DOWN, buff=1)
        ksp_full_matching.move_to(ksp_matching)
        ksp_full_regular.move_to(ksp_regular)

        self.play(Write(ksp_matching), Write(ksp_regular))

        self.play(
            TransformMatchingShapes(ksp_matching, ksp_full_matching),
            Transform(ksp_regular, ksp_full_regular),
        )

## Updatery

In [ ]:
%%manim -v WARNING -qh SimpleUpdaterExample


class SimpleUpdaterExample(Scene):
    def construct(self):
        square = Square()
        square_label = Tex("A neat square.").next_to(square, UP, buff=0.5)

        self.play(Write(square))
        self.play(FadeIn(square_label, shift=UP * 0.5))

        def label_updater(obj):
            """Updater, který posune objekt nad čtverec.

            První parametr (obj) je vždy objekt, na který je updater přidaný."""
            obj.next_to(square, UP, buff=0.5)

        # popisek čtverce chceme mít fixně nad čtvercem
        square_label.add_updater(label_updater)

        # vždy zůstává nad čtvercem
        self.play(square.animate.shift(LEFT * 3))
        self.play(square.animate.scale(1 / 2))
        self.play(square.animate.rotate(PI / 2).shift(RIGHT * 3 + DOWN * 0.5).scale(3))

        # odstranění updateru můžeme udělat přes remove_updater
        square_label.remove_updater(label_updater)
        self.play(square.animate.scale(1 / 3))
        self.play(square.animate.rotate(PI / 2))

In [ ]:
%%manim -v WARNING -qh BecomeUpdaterExample


class BecomeUpdaterExample(Scene):
    def format_point(self, point):
        """Formátování dané souřadnice na [x, y]."""
        return f"[{point[0]:.2f}, {point[1]:.2f}]"

    def construct(self):
        circle = Circle(color=WHITE)

        def circle_label_updater(obj):
            """Updater pro label, který jej posouvá nad bod a nastavuje jeho text."""
            obj.become(Tex(f"p = {self.format_point(circle.get_center())}"))
            obj.next_to(circle, UP, buff=0.35)

        self.play(Write(circle))

        circle_label = Tex()

        # používáme tu trochu trik k šetření kódu - updater voláme proto,
        # abychom nastavili popisek na základní hodnotu a pozici
        circle_label_updater(circle_label)

        self.play(FadeIn(circle_label, shift=UP * 0.3))

        circle_label.add_updater(circle_label_updater)

        # tato animace se bude pravděpodobně renderovat dlouho, protože
        # updater v každém snímku vytváří Tex objekt, což nějakou dobu trvá
        self.play(circle.animate.shift(RIGHT))
        self.play(circle.animate.shift(LEFT * 3 + UP))
        self.play(circle.animate.shift(DOWN * 2 + RIGHT * 2))
        self.play(circle.animate.shift(UP))

## Kostry úloh

### Trojúhelník [3b]

In [ ]:
%%manim -v WARNING -qh Triangle


class Triangle(Scene):
    def construct(self):
        pass

In [ ]:
%%manim -v WARNING -qh LineExample


class LineExample(Scene):
    def construct(self):
        p1 = Dot()
        p2 = Dot()
        
        points = VGroup(p1, p2).arrange(buff=2.5)
        
        line = Line(start=p1.get_center(), end=p2.get_center())
        
        self.play(Write(p1), Write(p2))
        
        self.play(Write(line))

In [ ]:
%%manim -v WARNING -qh CircleFromPointsExample


class CircleFromPointsExample(Scene):
    def construct(self):
        p1 = Dot().shift(LEFT + UP)
        p2 = Dot().shift(DOWN * 1.5)
        p3 = Dot().shift(RIGHT + UP)
        
        dots = VGroup(p1, p2, p3)
        
        # vytvoření kruhu ze tří bodů
        circle = Circle.from_three_points(p1.get_center(), p2.get_center(), p3.get_center(), color=WHITE)
        
        self.play(Write(dots), run_time=1.5)
        self.play(Write(circle))

### Vlna [6b]

In [ ]:
%%manim -v WARNING -qh Wave


class Wave(Scene):
    def construct(self):
        pass

In [ ]:
%%manim -v WARNING -qh ColorGradientExample


class ColorGradientExample(Scene):
    def construct(self):
        rows = 6
        square_count = rows * 9

        colors = ["#ef476f", "#ffd166", "#06d6a0", "#118ab2"]
        squares = [
            Square(fill_color=WHITE, fill_opacity=1).scale(0.3)
            for _ in range(square_count)
        ]

        group = VGroup(*squares).arrange_in_grid(rows=rows)

        self.play(Write(group, lag_ratio=0.04))

        all_colors = color_gradient(colors, square_count)

        self.play(
            AnimationGroup(
                *[s.animate.set_color(all_colors[i]) for i, s in enumerate(squares)],
                lag_ratio=0.02,
            )
        )

### Hilbert [6b]

In [ ]:
%%manim -v WARNING -qh Hilbert


class Path(VMobject):
    def __init__(self, points, *args, **kwargs):
        super().__init__(self, *args, **kwargs)
        self.set_points_as_corners(points)


class Hilbert(Scene):
    def construct(self):
        pass

In [ ]:
%%manim -v WARNING -qh PathExample


class Path(VMobject):
    def __init__(self, points, *args, **kwargs):
        super().__init__(self, *args, **kwargs)
        self.set_points_as_corners(points)


class PathExample(Scene):
    def construct(self):
        path = Path([LEFT + UP, LEFT + DOWN, RIGHT + UP, RIGHT + DOWN])

        self.play(Write(path))

        # znázornění bodů, ze kterých manim složil objekt
        # může se hodit při vytváření křivek
        path_points = VGroup(*[Dot().move_to(point) for point in path.points])
        
        self.play(Write(path_points))
        
        path2 = path.copy()
        path3 = path.copy()

        self.play(
            path2.animate.next_to(path, LEFT, buff=1),
            path3.animate.next_to(path, RIGHT, buff=1),
        )
        
        # pozor, tohle není úplně intuitivní
        # LEFT flipne dolů, jelikož určuje osu, přes kterou se objekt přetočí
        self.play(
            path2.animate.flip(),
            path3.animate.flip(LEFT),
        )